<a href="https://colab.research.google.com/github/Giftsboard/DataAnalysisProject/blob/main/COVID_19_Severity_and_Regional_Vulnerability_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Project Objective (Restated Clearly)**

We want to:

1. Measure observed COVID severity at province level

2. Estimate expected severity based on demographics & health capacity

3. Calculate excess severity (residuals)

4. Test whether macro-financial vulnerability explains excess severity

5. Visualise and classify provinces into risk typologies

# Import Libraries & Load Data

* pandas / numpy for data manipulation

* statsmodels for interpretable regression

* sklearn for clustering & scaling

* matplotlib / seaborn for publication-quality visuals

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

**Data Loading**

In [4]:
# Load Excel file (df) and do a quick structural scan to find key columns
import pandas as pd
import numpy as np

xlsx_path = 'unified_gfc_covid_hybrid.xlsx'

df = pd.read_excel(xlsx_path)

print(df.head())
print(df.columns.tolist()[:60])
print(df.shape)

# Basic column presence checks
key_checks = ['date','geo_type','country','region','province','daily_cases_avg7','daily_deaths_avg7','deceassed_per_100000',
              'hospitalized','intensive_care','2019_population','median_population_age','obesity','pop_density_2019',
              '2019_gdp_per_capita_(thousands)']
col_presence = pd.DataFrame({'col': key_checks, 'present': [c in df.columns for c in key_checks]})
print(col_presence)

   2019_gdp_per_capita_(thousands)  2019_population  \
0                              NaN              NaN   
1                              NaN              NaN   
2                              NaN              NaN   
3                              NaN              NaN   
4                              NaN              NaN   

   2019_real_gdp_(thousands)  2020_real_gdp_(millions)  activos  cases_14days  \
0                        NaN                       NaN      NaN           NaN   
1                        NaN                       NaN      NaN           NaN   
2                        NaN                       NaN      NaN           NaN   
3                        NaN                       NaN      NaN           NaN   
4                        NaN                       NaN      NaN           NaN   

   cases_7days  cases_accumulated  cases_accumulated_pcr  cases_pcr_14days  \
0          NaN                NaN                    NaN               NaN   
1          NaN            

**Data Cleaning, Manipulation, Filtering & Standardisation**

Basic data inspection plus focus on cleaning pipeline for province-level analysis.

Cleaned and structured dataset, focusing on province-level data by standardizing columns, parsing dates, and filtering relevant information.
- Created a copy of the raw DataFrame.
- Standardized string columns by stripping whitespace and replacing 'nan' and 'None' with NaN.
- Parsed the 'date' column to datetime format.
- Filtered the DataFrame to retain only province-level data if available.
- Built a list of columns to keep based on identifiers, COVID outcomes, structural controls, and financial vulnerability proxies.
- Coerced non-identifier columns to numeric types.
- Dropped rows without key identifiers and without dates.
- Removed duplicate rows.

In [9]:
raw_df = df.copy()

In [11]:
# Standardize obvious string columns
for col_name in ['geo_type','province','region','country','state','state_name','state_full','state_code','source','source_name']:
    if col_name in raw_df.columns:
        raw_df[col_name] = raw_df[col_name].astype(str).str.strip().replace({'nan': np.nan, 'None': np.nan})

In [12]:
# Parse date
if 'date' in raw_df.columns:
    raw_df['date'] = pd.to_datetime(raw_df['date'], errors='coerce')

In [13]:
# Prefer province level when available
if 'geo_type' in raw_df.columns:
    prov_df = raw_df[raw_df['geo_type'].astype(str).str.lower().eq('province')].copy()
else:
    prov_df = raw_df.copy()

In [14]:
# Build column allowlist by concept
id_cols = [c for c in ['date','country','region','province','ine_code','geo_type'] if c in prov_df.columns]

In [15]:
covid_outcome_cols = [c for c in prov_df.columns if any(k in c.lower() for k in [
    'daily_cases','daily_deaths','cases_','deaths_','deceased','recovered','hospitalized','intensive_care','icu','pcr','test','testac','num_casos'
])]

In [16]:
# Structural controls
struct_cols = [c for c in [
    '2019_population','poblacion','popdensity2018','pop_density_2019','median_population_age','obesity',
    'goveffectiv_2019','goveff_ch1907','deathrate_thous2018','temp_jan_2020','temperature',
    'icubeds_1','icubeds_2','icubeds_2_pub','hospitalized_per_100000','intensive_care_per_1000000'
] if c in prov_df.columns]

In [17]:
# Financial vulnerability proxies (GFC-style)
fin_cols = [c for c in prov_df.columns if any(k in c.lower() for k in [
    'cds','debt_growth','cb_','changecb','csm_','gap_gdp','real_gdp_growth','stockstestmax','gdpcapita','gdp_per_capita','real_gdp'
])]

In [18]:
keep_cols = []
for block in [id_cols, covid_outcome_cols, struct_cols, fin_cols]:
    for c in block:
        if c not in keep_cols:
            keep_cols.append(c)

In [19]:
focused_df = prov_df[keep_cols].copy()

In [22]:
# Coerce numeric columns (everything except identifiers)
id_set = set(id_cols)
for c in focused_df.columns:
    if c not in id_set:
        focused_df[c] = pd.to_numeric(focused_df[c], errors='coerce')

In [23]:
# Drop rows without key identifiers
if 'province' in focused_df.columns:
    focused_df = focused_df[focused_df['province'].notna()].copy()

In [24]:
# Drop rows without date if we want panel structure; otherwise keep latest snapshot per province later
if 'date' in focused_df.columns:
    focused_df = focused_df[focused_df['date'].notna()].copy()

In [25]:
# Remove exact duplicates
focused_df = focused_df.drop_duplicates().copy()

This code snippet identifies the most suitable geographic level for analysis based on the availability of non-null numeric data.
- Create a copy of the DataFrame and convert 'date' column to datetime.
- Check for 'geo_type' column and count occurrences of each geo level.
- Initialize a dictionary to store candidate frames based on geo levels.
- For each geo level, filter the DataFrame and check for the presence of an entity column.
- Count non-null numeric values in the filtered DataFrame.
- Store the results in the candidate frames dictionary.
- Select the geo level with the maximum number of non-null values or default to 'all' if none are found.

In [28]:
# If province-level filtered set is empty, fall back to best available geo level with real data.
raw_df = df.copy()
raw_df['date'] = pd.to_datetime(raw_df['date'], errors='coerce') if 'date' in raw_df.columns else pd.NaT

In [29]:
if 'geo_type' in raw_df.columns:
    geo_counts = raw_df['geo_type'].astype(str).str.lower().value_counts(dropna=False)
    geo_levels = geo_counts.index.tolist()
else:
    geo_levels = ['all']

In [30]:
geo_level_to_use = None
candidate_frames = {}

In [37]:
for lvl in geo_levels:
    if lvl == 'all':
        tmp = raw_df.copy()
    else:
        tmp = raw_df[raw_df['geo_type'].astype(str).str.lower().eq(str(lvl))].copy()
    # basic viability: has entity id and at least one numeric non-null outside ids
    entity_col = 'province' if 'province' in tmp.columns else ('country' if 'country' in tmp.columns else None)
    if entity_col is None:
        continue
    num_nonnull = 0
    for c in tmp.columns:
        if c in ['date','country','region','province','ine_code','geo_type']:
            continue
        num_nonnull += int(pd.to_numeric(tmp[c], errors='coerce').notna().sum())
    candidate_frames[str(lvl)] = {'rows': int(tmp.shape[0]), 'entity_col': entity_col, 'num_nonnull': int(num_nonnull)}

In [61]:
# pick level with max num_nonnull
if len(candidate_frames) > 0:
    geo_level_to_use = sorted(candidate_frames.items(), key=lambda kv: kv[1]['num_nonnull'], reverse=True)[0][0]
else:
    geo_level_to_use = 'all'
    geo_level_to_use
candidate_frames

{'all': {'rows': 0, 'entity_col': 'province', 'num_nonnull': 0}}

# Prepare Data For Analysis

In [62]:
work_df = df.copy()

In [63]:
# Standardize strings
for col_name in ['geo_type','province','region','country','state','state_name','state_full','state_code','source','source_name']:
    if col_name in work_df.columns:
        work_df[col_name] = work_df[col_name].astype(str).str.strip().replace({'nan': np.nan, 'None': np.nan})

In [64]:
# Parse date
if 'date' in work_df.columns:
    work_df['date'] = pd.to_datetime(work_df['date'], errors='coerce')

In [65]:
# Filter to best geo level discovered earlier
if 'geo_type' in work_df.columns and geo_level_to_use != 'all':
    analysis_df = work_df[work_df['geo_type'].astype(str).str.lower().eq(geo_level_to_use)].copy()
else:
    analysis_df = work_df.copy()

In [66]:
# Define allowlists for the structured analysis
id_cols = [c for c in ['date','country','region','province','ine_code','geo_type'] if c in analysis_df.columns]

In [67]:
# Outcomes and inputs for COVID severity construction
covid_cols = [c for c in [
    'daily_cases','daily_cases_avg7','daily_deaths','daily_deaths_avg7','daily_deaths_avg3','daily_deaths_inc',
    'cases_7days','cases_14days','cases_per_cienmil','cases_accumulated','cases_accumulated_pcr',
    'cases_pcr_7days','cases_pcr_14days','pcr','testac',
    'hospitalized','intensive_care','recovered','deceased','deceassed_per_100000',
    'num_casos','num_casos_prueba_pcr','num_casos_prueba_test_ac'
] if c in analysis_df.columns]

In [68]:
# Structural controls
struct_cols = [c for c in [
    '2019_population','poblacion','pop_density_2019','popdensity2018','median_population_age','obesity',
    'goveffectiv_2019','goveff_ch1907','deathrate_thous2018','temperature','temp_jan_2020',
    'icubeds_1','icubeds_2','icubeds_2_pub','hospitalized_per_100000','intensive_care_per_1000000',
    '2019_gdp_per_capita_(thousands)','gdpcapita_eurothous2019'
] if c in analysis_df.columns]


In [69]:
# Financial vulnerability proxies
fin_cols = [c for c in [
    'cb_100thous2007','changecb_100thous','cds_growth',
    'debt_growth_07-2010','debt_growth_07-2011','debt_growth_07-2012','debt_growth_07-2013',
    'csm_gap_gdp_potgdp','csm_gap_gdp_potgdp_long','csm_growthcds','csm_growthrealgdp',
    'real_gdp_growth_2007:q4_-_2009:q2(in%)','real_gdp_growth_2008:q1_-_2009:q2_(in%)'
] if c in analysis_df.columns]

In [70]:
keep_cols = []
for block in [id_cols, covid_cols, struct_cols, fin_cols]:
    for c in block:
        if c not in keep_cols:
            keep_cols.append(c)

In [71]:
analysis_df = analysis_df[keep_cols].copy()


In [72]:
# Coerce numeric for non-id
id_set = set(id_cols)
for c in analysis_df.columns:
    if c not in id_set:
        analysis_df[c] = pd.to_numeric(analysis_df[c], errors='coerce')

In [73]:
# Harmonize population
if '2019_population' in analysis_df.columns:
    analysis_df['pop'] = analysis_df['2019_population']
elif 'poblacion' in analysis_df.columns:
    analysis_df['pop'] = analysis_df['poblacion']
else:
    analysis_df['pop'] = np.nan

In [74]:
analysis_df['pop'] = pd.to_numeric(analysis_df['pop'], errors='coerce')

# Initialize base filter for 'province'
filter_condition = analysis_df['province'].notna()

# Add 'date' filter only if the 'date' column exists
if 'date' in analysis_df.columns:
    filter_condition = filter_condition & analysis_df['date'].notna()

analysis_df = analysis_df[filter_condition].copy()
analysis_df = analysis_df[(analysis_df['pop'].isna()) | (analysis_df['pop'] > 0)].copy()

In [75]:
# Drop rows with no COVID info at all (to keep real numbers)
covid_nonnull = None
if len(covid_cols) > 0:
    covid_nonnull = analysis_df[covid_cols].notna().any(axis=1)
    analysis_df = analysis_df[covid_nonnull].copy()

In [76]:
# Keep 2020-2021 window (typical for COVID severity waves)
if 'date' in analysis_df.columns and not analysis_df.empty:
    analysis_df = analysis_df[(analysis_df['date'] >= pd.Timestamp('2020-01-01')) & (analysis_df['date'] <= pd.Timestamp('2021-12-31'))].copy()

In [81]:
# Remove duplicates per province-date by taking max of numeric fields (common in merged sources)
numeric_cols = [c for c in analysis_df.columns if c not in id_set and c not in ['pop']]
agg_map = {c: 'max' for c in numeric_cols}
for c in id_cols:
    agg_map[c] = 'first'
agg_map['pop'] = 'max'

In [82]:
analysis_df = (analysis_df.sort_values('date')
               .groupby(['province','date'], dropna=False, as_index=False)
               .agg(agg_map))


KeyError: 'date'